# 在Intel GPU上使用IPEX-LLM进行本地嵌入

> [IPEX-LLM](https://github.com/intel-analytics/ipex-llm/)是一个用于在Intel CPU和GPU上运行LLM的PyTorch库（例如，带有iGPU的本地PC，离散GPU，如Arc、Flex和Max），具有非常低的延迟。

本示例介绍了如何使用LlamaIndex在Intel GPU上进行嵌入任务，以及如何使用`ipex-llm`优化。这对于RAG、文档QA等应用非常有帮助。

> **注意**
>
> 您可以参考[这里](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/embeddings/llama-index-embeddings-ipex-llm/examples)获取`IpexLLMEmbedding`的完整示例。请注意，在Intel GPU上运行时，请在运行示例时在命令参数中指定`-d 'xpu'`。

## 安装先决条件
要在Intel GPU上受益于IPEX-LLM，需要进行一些先决条件的工具安装和环境准备步骤。

如果您是Windows用户，请访问[在带有Intel GPU的Windows上安装IPEX-LLM指南](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Quickstart/install_windows_gpu.html)，并按照[**安装先决条件**](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Quickstart/install_windows_gpu.html#install-prerequisites)更新GPU驱动程序（可选）并安装Conda。

如果您是Linux用户，请访问[在带有Intel GPU的Linux上安装IPEX-LLM](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Quickstart/install_linux_gpu.html)，并按照[**安装先决条件**](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Quickstart/install_linux_gpu.html#install-prerequisites)安装GPU驱动程序、Intel® oneAPI Base Toolkit 2024.0和Conda。

## 安装`llama-index-embeddings-ipex-llm`

完成先决条件安装后，您应该已经创建了一个包含所有先决条件的conda环境，激活您的conda环境并安装`llama-index-embeddings-ipex-llm`如下所示：

```bash
conda activate <your-conda-env-name>

pip install llama-index-embeddings-ipex-llm[xpu] --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/xpu/us/
```
此步骤还将安装`ipex-llm`及其依赖项。

> **注意**
>
> 您也可以使用`https://pytorch-extension.intel.com/release-whl/stable/xpu/cn/`作为`extra-indel-url`。

## 运行时配置

为了获得最佳性能，建议根据您的设备设置几个环境变量：

### 对于带有Intel Core Ultra集成GPU的Windows用户

在Anaconda Prompt中：

```
set SYCL_CACHE_PERSISTENT=1
set BIGDL_LLM_XMX_DISABLED=1
```

### 对于带有Intel Arc A-Series GPU的Linux用户

```bash
# 配置oneAPI环境变量。对于APT或离线安装的oneAPI，这是必需的步骤。
# 对于通过PIP安装的oneAPI，由于环境已经在LD_LIBRARY_PATH中配置，因此可以跳过此步骤。
source /opt/intel/oneapi/setvars.sh

# 建议的环境变量以获得最佳性能
export USE_XETLA=OFF
export SYCL_PI_LEVEL_ZERO_USE_IMMEDIATE_COMMANDLISTS=1
export SYCL_CACHE_PERSISTENT=1
```

> **注意**
>
> 每次在Intel iGPU/Intel Arc A300-Series或Pro A60上首次运行每个模型时，可能需要几分钟来编译。
>
> 对于其他GPU类型，请参考[这里](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Overview/install_gpu.html#runtime-configuration)（Windows用户）和[这里](https://ipex-llm.readthedocs.io/en/latest/doc/LLM/Overview/install_gpu.html#id5)（Linux用户）。

## `IpexLLMEmbedding`

在初始化`IpexLLMEmbedding`时设置`device="xpu"`将在Intel GPU上放置嵌入模型，并受益于IPEX-LLM优化：

```python
from llama_index.embeddings.ipex_llm import IpexLLMEmbedding

embedding_model = IpexLLMEmbedding(
    model_name="BAAI/bge-large-en-v1.5", device="xpu"
)
```

> 请注意，`IpexLLMEmbedding`目前仅为Hugging Face Bge模型提供优化。

然后，您可以像平常一样进行嵌入任务：

```python
sentence = "IPEX-LLM is a PyTorch library for running LLM on Intel CPU and GPU (e.g., local PC with iGPU, discrete GPU such as Arc, Flex and Max) with very low latency."
query = "What is IPEX-LLM?"

text_embedding = embedding_model.get_text_embedding(sentence)
print(f"embedding[:10]: {text_embedding[:10]}")

text_embeddings = embedding_model.get_text_embedding_batch([sentence, query])
print(f"text_embeddings[0][:10]: {text_embeddings[0][:10]}")
print(f"text_embeddings[1][:10]: {text_embeddings[1][:10]}")

query_embedding = embedding_model.get_query_embedding(query)
print(f"query_embedding[:10]: {query_embedding[:10]}")
```
